In [1]:
import scipy.io
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy.interpolate import interp1d
from joblib import load,dump

import sys
sys.path.insert(0,'../scripts/')
from processDynamic import processDynamic

In [2]:
# # I saved my own model data as pickle files, but may not need
# model_output = load('../data/p14model-ocv.pkl')
# model_data = load('../data/p14model-data.pkl')

In [3]:
# Note, had to resave this data in Octave like so: save -mat7-binary 'file.mat' model
#data = scipy.io.loadmat('../data/P14model-ocv-mat7.mat', simplify_cells=True)

In [4]:
#data['model']['model'].keys()

In [5]:
test = processDynamic(model_dir='../data/')

In [6]:
test.load_data()

In [7]:
#plt.plot(test.data[5]['script1']['time'],test.data[5]['script1']['current'])

In [8]:
test.process_DYN_step1()

25 14.531852290290429
5 14.482598404517704
45 14.414656461803379


In [9]:
test.process_DYN_step2()

ValueError: shape mismatch: value array of shape (33460,) could not be broadcast to indexing result of shape (11532,)

In [11]:
test.model[5]

{'Q': 14.482598404517704, 'eta': 0.9874053548391215}

In [10]:
test.data[5]

{'script1': {'rawTime': array([   60.00760673,   120.01999502,   180.03245408, ...,
         42458.93850697, 42459.95243118, 42460.84167311]),
  'time': array([ 6901.0185148,  6902.0185148,  6903.0185148, ..., 42458.0185148,
         42459.0185148, 42460.0185148]),
  'rawStep': array([1, 1, 1, ..., 8, 8, 8], dtype=uint8),
  'step': array([1, 1, 1, ..., 8, 8, 8], dtype=uint8),
  'rawCurrent': array([0., 0., 0., ..., 0., 0., 0.]),
  'current': array([-0., -0., -0., ..., -0., -0., -0.]),
  'rawVoltage': array([4.14778471, 4.14778471, 4.14778471, ..., 3.34867883, 3.34867883,
         3.34884071]),
  'voltage': array([4.14746094, 4.14746094, 4.14746094, ..., 3.34882571, 3.34867883,
         3.34869086]),
  'rawChgAh': array([ 0.        ,  0.        ,  0.        , ..., 11.84536513,
         11.84536513, 11.84536513]),
  'chgAh': array([ 0.        ,  0.        ,  0.        , ..., 11.84536513,
         11.84536513, 11.84536513]),
  'rawDisAh': array([ 0.        ,  0.        ,  0.        , ...,